In [9]:
# generate csv from data 

import csv

header = ['phrase_id', 'phrase', 'sentiment_score', 'implication']

f_dict = open('./data/english/dictionary.txt', 'r')
f_sent = open('./data/english/sentiment_labels.txt', 'r')

f = open('./data/english/sentiment_data.csv', 'w')
writer = csv.writer(f)
writer.writerow(header)

lines_dict = f_dict.readlines()
lines_sent = f_sent.readlines()

for line in lines_dict:

    data = []
    line_dict_list = line.strip().split('|')
    sentiment_val = lines_sent[int(line_dict_list[1])+1].split('|')[1]
    sentiment_val = float(sentiment_val)
    implication = ""

    if sentiment_val<=0.2:
        implication = "very_negative"
    elif sentiment_val<=0.4:
        implication = "negative"
    elif sentiment_val<=0.6:
        implication = "neutral"
    elif sentiment_val<=0.8:
        implication = "positive"
    else:
        implication = "very_positive"

    data.append(line_dict_list[1])
    data.append(line_dict_list[0])
    data.append(sentiment_val)
    data.append(implication)

    writer.writerow(data)

f_dict.close()
f_sent.close()
f.close()


In [13]:
# remove urls
import pandas as pd  

df = pd.read_csv('./data/english/sentiment_data.csv')    

df['phrase'] = df['phrase'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

df.to_csv('./data/english/sentiment_data_url_removed.csv')


In [7]:
# remove stop words
# https://stackabuse.com/python-for-nlp-working-with-the-gensim-library-part-1/

from gensim.parsing.preprocessing import remove_stopwords
import pandas as pd  

df = pd.read_csv('./data/english/sentiment_data_url_removed.csv')

for index, phrase in df['phrase'].iteritems():
    df['phrase'][index] = remove_stopwords(phrase)

df.to_csv('./data/english/sentiment_data_url_stopwords_removed.csv')



In [8]:
# lowercase all words
import pandas as pd  

df = pd.read_csv('./data/english/sentiment_data_url_removed.csv')

df['phrase'] = [each_string.lower() for each_string in df['phrase']]

df.to_csv('./data/english/sentiment_data_url_stopwords_removed_lower.csv')
